# Building a model based on financial ratios

https://towardsdatascience.com/building-an-investing-model-with-python-44d779adb6bf

# Overview
- Investment model to find attractive stocks based on financial ratios
- Find all technology  stocks in Nasdaq exchange
- Compute main financial ratios for each stock
- Based on a custom ranking criteria, build a model to find the most attractive stocks

# Approach
- Use financialmodelingprep API  
- Make a Get request to the financial API to extract Nasdaq ticker
- Filter by Technology sector and bigger than a particular size
- Loop through each company and extract the financial ratios
- Append each of the ratios for each of the companies into a Python dictionary and create a Pandas DataFrame
- Set up an investment model ranking criteria and come up with a list of most attractive stocks according to model

In [15]:
import requests
import pandas as pd

In [22]:
# API_Key = "33a7c94e046defe591808732c7e21281"
api_key = '33a7c94e046defe591808732c7e21281'
companies = []
marketcap = str(10000000000)
volumecap = str(10000000)

In [23]:
url = (f'https://financialmodelingprep.com/api/v3/stock-screener?marketCapMoreThan={marketcap}&betaMoreThan=1&volumeMoreThan=10000&sector=Technology&exchange=NASDAQ&dividendMoreThan=0&limit={volumecap}&apikey={api_key}')

In [24]:
#get companies based on criteria defined about
screener = requests.get(url).json()

#print(screener)

In [25]:
for item in screener:
    companies.append(item['symbol'])

In [26]:
print(companies)

['AAPL', 'MSFT', 'NVDA', 'ADBE', 'CSCO', 'ASML', 'TXN', 'QCOM', 'INTU', 'AMD', 'AMAT', 'MU', 'ADSK', 'LRCX', 'TEAM', 'WDAY', 'ADI', 'NXPI', 'CTSH', 'SPLK', 'KLAC', 'SNPS', 'CDNS', 'MCHP', 'OKTA', 'ANSS', 'XLNX', 'MRVL', 'VRSN', 'SWKS', 'FTNT', 'COUP', 'TTD', 'MXIM', 'CDW', 'ZS', 'TER', 'ZBRA', 'SSNC', 'WIX', 'QRVO', 'SYMC', 'WDC', 'GDS', 'STX', 'NLOK', 'LOGI', 'MPWR', 'TRMB', 'CGNX', 'ULTI']


# Compute Financial Ratios
- Next, we extract all financial ratios from the Nasdaq companies that we obtained in the previous section. 
- For our investment model, we will work with below selected financial ratios.

In [28]:
value_ratios ={}
#get the financial ratios
count = 0
for company in companies:
    try:
        if count <10:
            count = count + 1
            fin_ratios = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{company}?apikey={api_key}').json()
            value_ratios[company] = {}
            value_ratios[company]['ROE'] = fin_ratios[0]['returnOnEquity']
            value_ratios[company]['ROA'] = fin_ratios[0]['returnOnAssets']
            value_ratios[company]['Debt_Ratio'] = fin_ratios[0]['debtRatio']
            value_ratios[company]['Interest_Coverage'] = fin_ratios[0]['interestCoverage']
            value_ratios[company]['Payout_Ratio'] = fin_ratios[0]['payoutRatio']
            value_ratios[company]['Dividend_Payout_Ratio'] = fin_ratios[0]['dividendPayoutRatio']
            value_ratios[company]['PB'] = fin_ratios[0]['priceToBookRatio']
            value_ratios[company]['PS'] = fin_ratios[0]['priceToSalesRatio']
            value_ratios[company]['PE'] = fin_ratios[0]['priceEarningsRatio']
            value_ratios[company]['Dividend_Yield'] = fin_ratios[0]['dividendYield']
            value_ratios[company]['Gross_Profit_Margin'] = fin_ratios[0]['grossProfitMargin']
       #more financials on growth:https://financialmodelingprep.com/api/v3/financial-growth/AAPL?apikey=demo
            growth_ratios = requests.get(f'https://financialmodelingprep.com/api/v3/financial-growth/{company}?apikey={demo}').json()
            value_ratios[company]['Revenue_Growth'] = growth_ratios[0]['revenueGrowth']
            value_ratios[company]['NetIncome_Growth'] = growth_ratios[0]['netIncomeGrowth']
            value_ratios[company]['EPS_Growth'] = growth_ratios[0]['epsgrowth']
            value_ratios[company]['RD_Growth'] = growth_ratios[0]['rdexpenseGrowth']
        
    except:
          pass

#print(value_ratios)

In [31]:
#print(value_ratios)

In [30]:
DF = pd.DataFrame.from_dict(value_ratios,orient='index')
print(DF.head(4))

           ROE       ROA  Debt_Ratio  Interest_Coverage  Payout_Ratio  \
AAPL  0.610645  0.163230    0.732692          18.382830      0.255520   
MSFT  0.383465  0.136937    0.642897           5.881981      0.351962   
NVDA  0.229105  0.161478    0.295178          57.115385      0.139485   
ADBE  0.280286  0.142154    0.492826          20.384578      0.000000   

      Dividend_Payout_Ratio         PB         PS         PE  Dividend_Yield  \
AAPL               0.255520  12.709658   4.420394  20.813515        0.012277   
MSFT               0.351962  10.523850   8.557532  27.444076        0.012825   
NVDA               0.139485  13.076733  14.617004  57.077414        0.002444   
ADBE                    NaN  15.276147  14.399420  54.501944             NaN   

      Gross_Profit_Margin  Revenue_Growth  NetIncome_Growth  EPS_Growth  \
AAPL             0.378178       -0.020411         -0.071811   -0.003331   
MSFT             0.659020        0.140295          1.367992    1.376744   
NVDA    

In [32]:
#criteria ranking
ROE = 1.2
ROA = 1.1
Debt_Ratio = -1.1
Interest_Coverage = 1.05
Dividend_Payout_Ratio = 1.01
PB = -1.10
PS = -1.05
Revenue_Growth = 1.25
Net_Income_Growth = 1.10

In [33]:
#mean to enable comparison across ratios
ratios_mean = []
for item in DF.columns:
    ratios_mean.append(DF[item].mean())
#divide each value in dataframe by mean to normalize values
DF = DF / ratios_mean

In [34]:
DF['ranking'] = DF['NetIncome_Growth']*Net_Income_Growth + \
                DF['Revenue_Growth']*Revenue_Growth  + \
                DF['ROE']*ROE + DF['ROA']*ROA + \
                DF['Debt_Ratio'] * Debt_Ratio + \
                DF['Interest_Coverage'] * Interest_Coverage + \
                DF['Dividend_Payout_Ratio'] * Dividend_Payout_Ratio + \
                DF['PB']*PB + DF['PS']*PS

In [35]:
print(DF.sort_values(by=['ranking'],ascending=False))

           ROE       ROA  Debt_Ratio  Interest_Coverage  Payout_Ratio  \
CSCO  0.847394  0.751661    1.188703           0.806260      1.803891   
QCOM  2.187166  0.841797    1.540463           0.567115      2.372578   
MSFT  0.938712  0.866176    1.163693           0.279578      1.234017   
TXN   1.378856  1.761262    0.915286           1.601522      2.102127   
AAPL  1.494840  1.032492    1.326229           0.873758      0.895880   
NVDA  0.560843  1.021413    0.534294           2.714763      0.489049   
ADBE  0.686133  0.899178    0.892053           0.968903      0.000000   
ASML  0.594108  0.900696    0.748065                NaN     -0.025711   
INTU  1.016669  1.567502    0.730023                NaN      1.128169   
AMD   0.295281  0.357823    0.961191           0.188102      0.000000   

      Dividend_Payout_Ratio        PB        PS        PE  Dividend_Yield  \
CSCO               1.443113  0.452428  0.476968  0.390849        1.938229   
QCOM               1.898062  1.468826  0.4